In [1]:
# Determining how to apply actions to Go2 joints
# Reference
# https://genesis-world.readthedocs.io/en/latest/user_guide/getting_started/control_your_robot.html#joint-control
# https://github.com/Genesis-Embodied-AI/Genesis/tree/main/examples/locomotion

# Importing libraries
import genesis as gs

#Initializes Genesis with the CPU backend.
gs.init(backend=gs.cpu)

#Create a Scene
scene = gs.Scene(show_viewer=True)

#Adds a flat ground plane to the scene.
plane = scene.add_entity(gs.morphs.Plane())

#Integrate the Go2 Robot xml.
robot = gs.morphs.MJCF(file="xml/Unitree_Go2/go2.xml")

#Add an entity to the scene.
Go2 = scene.add_entity(robot)

#Builds the scene.
scene.build()

[Genesis] [03:19:48] [INFO] ╭─────────────────────────────────────────────────────────────────────────────────────╮
[Genesis] [03:19:48] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉│
[Genesis] [03:19:48] [INFO] ╰─────────────────────────────────────────────────────────────────────────────────────╯
[Genesis] [03:19:50] [INFO] Running on [Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz] with backend gs.cpu. Device memory: 15.78 GB.
[Genesis] [03:19:50] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [03:19:51] [INFO] Scene <b63d1fb> created.
[Genesis] [03:19:51] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <317e73a>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [03:19:51] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <8f9e1f4>, morph: <gs.morphs.MJCF(file='C:\Users\ritwi\Desktop\total_robotics\genesis_AI_sims\Unitree_Go2\xml\Unitree_Go2\go2.xml')>, m

In [2]:
# Declaring the joint names in the desired order
joint_names = [
    'FL_hip_joint',
    'FR_hip_joint',
    'RL_hip_joint',
    'RR_hip_joint',
    'FL_thigh_joint',
    'FR_thigh_joint',
    'RL_thigh_joint',
    'RR_thigh_joint',
    'FL_calf_joint',
    'FR_calf_joint',
    'RL_calf_joint',
    'RR_calf_joint', 
]

# Defining the Joint Name IDs
motors_dof_idx = [Go2.get_joint(name).dof_start for name in joint_names]
# Printing the Joint Name IDs
motors_dof_idx

[0, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [3]:
import numpy as np

# Define the proportional (kp) and derivative (kd) gain values
kp= 20.0
kd= 0.5

# Create an array of kp values for each motor degree of freedom (DOF)
kp_array = np.full(len(motors_dof_idx), kp)
# Print the kp array to verify its values
print('Proportional Gains Array:\n',kp_array)

# Create an array of kd values for each motor degree of freedom (DOF)
kd_array = np.full(len(motors_dof_idx), kd)
# Print the kd array to verify its values
print('Derivative Gains Array:\n',kd_array)

print('\nSetting Proportional and Derivative gains for each DOF...')
# Set the proportional gains (Kp) for the specified motor joints (motors_dof_idx)
Go2.set_dofs_kp(kp_array, motors_dof_idx)

# Set the derivative gains (Kv/Kd) for the specified motor joints (motors_dof_idx)
Go2.set_dofs_kv(kd_array, motors_dof_idx)

print('\nThe Proportional and Derivative gains of the DOFs are set')

print('\nRechecking the Proportional and Derivative gains for each DOF...')
print('\nThe Proportional gains of the DOFs are as follows')
print(Go2.get_dofs_kp(motors_dof_idx))
print('\nThe Proportional gains of the DOFs are as follows')
print(Go2.get_dofs_kv(motors_dof_idx))

Proportional Gains Array:
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
Derivative Gains Array:
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]

Setting Proportional and Derivative gains for each DOF...

The Proportional and Derivative gains of the DOFs are set

Rechecking the Proportional and Derivative gains for each DOF...

The Proportional gains of the DOFs are as follows
tensor([20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.])

The Proportional gains of the DOFs are as follows
tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000])


In [4]:
# There exist the following classes of joints
# 1. class abduction joints (all 4 hips): FL_hip_joint, FR_hip_joint, RL_hip_joint, RR_hip_joint, Range= [-1.0472, 1.0472]
# 2. class front_hip joints (2 front thighs): FL_thigh_joint, FR_thigh_joint, Range= [-1.5708 3.4907]
# 3. class back_hip joints (2 rear thighs): RL_thigh_joint, RR_thigh_joint, Range= [-0.5236 4.5379]
# 4. class knee (all 4 calfs): FL_calf_joint, FR_calf_joint, RL_calf_joint, RR_calf_joint, Range= [-2.7227 -0.83776]

import gym

# Joint position limits in radians (based on the Go2 xml)
action_low = np.array([-1.0472, -1.0472, -1.0472, -1.0472, -1.5708, -1.5708, -0.5236, -0.5236, -2.7227, -2.7227, -2.7227, -2.7227], dtype=np.float32)
action_high = np.array([1.0472, 1.0472, 1.0472, 1.0472, 3.4907, 3.4907, 4.5379, 4.5379, -0.83776, -0.83776, -0.83776, -0.83776], dtype=np.float32)

# Create the action space
action_space = gym.spaces.Box(low=action_low, high=action_high, dtype=np.float32)

# Sample a random action
action = action_space.sample()
print("Sampled action:\n", action)

Sampled action:
 [ 0.73290414 -0.22990401  0.9629305   0.0737622  -0.57484674 -0.90785503
  0.8568048   1.4575216  -1.1919217  -1.6458828  -2.0015185  -1.7111005 ]


In [ ]:
# Initial position
initial_position = np.array([0.0, 0.8, -1.5,   # FL_hip, FL_thigh, FL_calf
                             0.0, 0.8, -1.5,   # FR_hip, FR_thigh, FR_calf
                             0.0, 0.8, -1.5,   # RL_hip, RL_thigh, RL_calf
                             0.0, 0.8, -1.5    # RR_hip, RR_thigh, RR_calf
                             ], dtype=np.float32)

# Hard reset the initial position
for i in range(10):
    Go2.set_dofs_position(initial_position, motors_dof_idx)
    scene.step()

# Try random actions using the control_dofs_position command
for i in range(100):
    action = action_space.sample()
    clipped_action = np.clip((initial_position+ action), action_low, action_high)
    print(clipped_action)
    Go2.control_dofs_position(clipped_action, motors_dof_idx)
    
    for i in range(10):
        scene.step()


[Genesis] [03:24:57] [INFO] Running at 0.40 FPS.
[Genesis] [03:24:57] [INFO] Running at 0.42 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.44 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.46 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.48 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.51 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.54 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.56 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.59 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.62 FPS.
[ 1.040954   -0.18818784 -1.0472      0.46534798  2.7025356  -0.17069244
  0.97772855  4.5379     -2.7227     -2.5729234  -0.83776    -2.7227    ]
[Genesis] [03:24:58] [INFO] Running at 0.65 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.69 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.72 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.76 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.80 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.84 FPS.
[Genesis] [03:24:58] [INFO] Running at 0.88 FPS.
[Genesis] [03:24:58]